# Let's get started with RunGPU

Rungpu is simple to use. You can finetune your model in a few lines of code.  
A few basics: 

1. Pick your dataset of your choice. 
2. Pick any Model from Huggingface. 
3. Build your own finetuning configuration. Or choose from one of our templates!

We have sample code to help you get started, with popular models, including Mistral v0.2, Llama 3, Gemma, etc.

Let's start with importing the libraries.

In [6]:
# Lets import the model first
from rungpu import Finetune


## Create a Client object for Authentication. 


In [8]:
# Import the Client class to get access to RunGPU services. 
from rungpu import Client
# Enter your Unique RunGPU Client ID and Secret here. 
client_id = '<Your RunGPU Client ID>'
client_secret = '<Your RunGPU Client Secret>'

client = Client(client_id, client_secret)

## Let's get started with creating your dataset. 

### Why? 

In this example, we will use a dataset to train the Chatbot Model to make sure it learns specific information about our use case. 


But lets start with a simpler question:

#### What is a dataset and why do you need it? 

A dataset is simply a document or simply a corpus of text that holds the knowledge you would want to train your chatbot on. 
It could be anything under the sun, general or specific. Since most foundational models are trained on a lot of general knowledge, 
it would most likely be something specific. 




A typical dataset config would look like the following
For example, for a file in aws S3, 
```
{
    "config": {
        "type": "s3",
        "provider": "AWS",
        "env_auth": "false",
        "access_key_id": <access_key_id for aws>,
        "secret_access_key": <secret access key>,
        "region": <aws region>,
        "src_file": "/rungpu-dev/test.txt",
        "dest_file": "data.txt"
    }
}
```

the `config` property is where you'll be entering all your config information for where your file will be coming from. 




In [9]:
aws_access_key_id = config['aws']['AWS_ACCESS_KEY_ID']
aws_secret_access_key = config['aws']['AWS_SECRET_ACCESS_KEY']
aws_region=config['aws']['AWS_REGION']


dataset_config = {
    "config": {
        "type": "s3",
        "provider": "AWS",
        "env_auth": "false",
        "access_key_id": aws_access_key_id,
        "secret_access_key": aws_secret_access_key,
        "region": aws_region,
        "src_file": "/rungpu-dev/test.txt",

    }
}



In [10]:
import json
from rungpu import Dataset

config_path = "./dataset.json"
gdrive_url= "https://drive.google.com/file/d/1zj8v7Nxf2gZuScLs8hzcu-jSBYkOP-no/view?usp=sharing"
# Change to json objects , not files. 
gdrive_config = {'config':{'type':'google_drive','src_file':gdrive_url}}


dataset = Dataset(client=client,mode='train', config=gdrive_config)

# Pulling the Dataset from the cloud. 
dataset_response = dataset.create_dataset()


In [11]:
dataset_response

{'command': 'Create Dataset',
 'created_at': '2024-07-24 14:53:48.179637',
 'dataset_id': 'rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c',
 'src_file': 'https://drive.google.com/file/d/1zj8v7Nxf2gZuScLs8hzcu-jSBYkOP-no/view?usp=sharing',
 'data_source': 'google_drive'}

# Your Dataset ID.

Running the code cell above returns you a response which contains the details of the dataset you just created. 

The dataset you created is stored securely on our servers for the purposes of finetuning the model (it can be removed later.)

This response object you get on running the `create_dataset()` function contains the dataset_id unique to the dataset you just created. The dataset id looks something like the following: 

`rungpu_dataset_<random_unique_id>`

You can plug this id into your finetuning config as below for the `dataset_id` property

In [12]:
dataset_id = dataset_response['dataset_id']
dataset_id

'rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c'

# Let's Create your model and start Finetuning!

Your finetuning job flow includes the creation of the model. So the finetuning config would include the config for your model. 

The finetuning config looks something like this: 

```
    {
    "base_model": <huggingface base model>,
    "quant": 8,
    "num_steps": 100,
    "dataset_id": <rungpu dataset id will be added by the Finetune Object when you declare it.>,
    "strategy": "lora",
    "checkpoint_steps": 10,
    "training_size": 1000,
    "peft_config": {
        "lora": {
            "r": 16,
            "alpha": 16,
            "target_modules": [
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "gate_proj",
                "up_proj",
                "down_proj",
                "lm_head"
            ],
            "bias": "none",
            "lora_dropout": 0.05
        }
    }
}
```


In [13]:
# Here's your finetuning config. 
ft_config = {
    "base_model": "mistralai/Mistral-7B-Instruct-v0.2",
    "quant": 8,
    "num_steps": 10,
    "strategy": "lora",
    "checkpoint_steps": 1,
    "training_size": 1000,
    "model_max_length": 4096,
    "prompt_max_length": 512,
    "gguf_flag": True,
    "peft_config": {
        "lora": {
            "r": 16,
            "alpha": 16,
            "target_modules": [
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "gate_proj",
                "up_proj",
                "down_proj",
                "lm_head"
            ],
            "bias": "none",
            "lora_dropout": 0.05
        }
    }
}

In [14]:
# Define the finetuning config
import json

# Getting the config file
config_path = './ft_config.json'

# Remove files. Just json objects. 

# Creating the Finetune object, and plugging in the config we created, finetuning specific. 
finetune = Finetune(client,dataset_id=dataset_id,config=ft_config)

#Calling the run_finetune() function to kick off the finetuning job. 
response = finetune.run_finetune()
response


{'command': 'Finetune',
 'status': 'IN_QUEUE',
 'train_id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc-rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c',
 'base_model': 'mistralai/Mistral-7B-Instruct-v0.2',
 'quantization': 8,
 'dataset_id': 'rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c',
 'peft_strategy': 'lora',
 'checkpoints': 1,
 'num_steps': 10,
 'training_size': 1000,
 'peft_config': {'lora': {'r': 16,
   'alpha': 16,
   'target_modules': ['q_proj',
    'k_proj',
    'v_proj',
    'o_proj',
    'gate_proj',
    'up_proj',
    'down_proj',
    'lm_head'],
   'bias': 'none',
   'lora_dropout': 0.05}}}

# The Finetune `run_id`

Once you execute the `run_finetune()` function on the finetune object, you essentially kick off a finetuning object on the backend. 
the `run_finetune()` call returns a response object from the server, that contains a `run_id`, which is the unique identifier for this specific finetuning job run. 

You can use this run_id to retrieve the details on how the run job progressed, and other information about model configurations and finetuning strategies. This is put in place with several other functions we provide to check the finetune job details. 


The response of the `run_finetune()` command looks something like the following.

In [15]:
response
train_id = response['train_id']

### Let's see what our training id looks like

The training id is a combination of the model_id and the training dataset id that it was trained on. This gives your training job/ finetuned model a unique identity. 

In [16]:
train_id

'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc-rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c'

# Check the status of your run. 

Use the finetune object to call the `get_status` function, which will help us get run status of current or past finetuning run jobs based on their `run_id`

In [17]:
from rungpu import TrainStatus, Client
client = Client(client_id=client_id, client_secret=client_secret)
train_status = TrainStatus(client,train_id)
status = train_status.get_status()
status

{'Train_Id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc-rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c', 'time_elapsed': 'null', 'train_status': {'command': 'Finetune', 'train_id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc-rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c', 'status': 'IN QUEUE', 'phase': 'TRAINING', 'client_id': 'n6p7iWSrknJqkLIwX0PGi', 'model_id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc', 'base_model': 'mistralai/Mistral-7B-Instruct-v0.2', 'dataset_id': 'rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c', 'run_start': 'null', 'run_end': 'null', 'export_start': 'null', 'export_end': 'null', 'error': 'Job Still in Queue', 'quantization': 8, 'strategy': 'lora', 'checkpoint_steps': 1, 'training_steps': 10, 'training_split': 1000, 'peft_config': {'lora': {'r': 16, 'alpha': 16, 'target_modules': ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_p

{'Train_Id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc-rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c',
 'time_elapsed': 'null',
 'train_status': {'command': 'Finetune',
  'train_id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc-rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c',
  'status': 'IN QUEUE',
  'phase': 'TRAINING',
  'client_id': 'n6p7iWSrknJqkLIwX0PGi',
  'model_id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc',
  'base_model': 'mistralai/Mistral-7B-Instruct-v0.2',
  'dataset_id': 'rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c',
  'run_start': 'null',
  'run_end': 'null',
  'export_start': 'null',
  'export_end': 'null',
  'error': 'Job Still in Queue',
  'quantization': 8,
  'strategy': 'lora',
  'checkpoint_steps': 1,
  'training_steps': 10,
  'training_split': 1000,
  'peft_config': {'lora': {'r': 16,
    'alpha': 16,
    'target_modules': ['q_proj',
     'k_proj',


In [20]:
train_status.progress()



{'Train_Id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc-rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c', 'time_elapsed': 'null', 'train_status': {'command': 'Finetune', 'train_id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc-rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c', 'status': 'IN QUEUE', 'phase': 'TRAINING', 'client_id': 'n6p7iWSrknJqkLIwX0PGi', 'model_id': 'mistralai-Mistral-7B-Instruct-v0.2-8-bit-b78fc644-4978-11ef-8748-b8ca3a5c98fc', 'base_model': 'mistralai/Mistral-7B-Instruct-v0.2', 'dataset_id': 'rungpu_dst_7882f5b6-fce1-416d-8bf2-289303761a9c', 'run_start': 'null', 'run_end': 'null', 'export_start': 'null', 'export_end': 'null', 'error': 'Job Still in Queue', 'quantization': 8, 'strategy': 'lora', 'checkpoint_steps': 1, 'training_steps': 10, 'training_split': 1000, 'peft_config': {'lora': {'r': 16, 'alpha': 16, 'target_modules': ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_p